# Different possibilities
There is different possibilities out there:

- Use WikiData: contains a lot of data from Wikipedia (and other) in a relational style. However, data about politician/big swiss companies aren't part of the Wikidata
- `Wikipedia` python module (or, more generally, the MediaWiki API) where one can find more structured information. However, in this case, most of the features aren't useful (at least at first)
- Scrape page directly. This solution will be tried first

# Scraping Wikipedia

In [12]:
import requests                       # HTTP requests lib
from bs4 import BeautifulSoup         # HTML parsing
import pandas as pd                   # Data handling
import lxml

WIKIPEDIA_URL = "https://en.wikipedia.org/wiki/List_of_Swiss_companies_by_revenue"

page = BeautifulSoup(requests.get(WIKIPEDIA_URL).content, "html.parser")
#print(page.prettify())

content_div = page.find('div', {'id':'mw-content-text'}) 

table_class_target = set('wikitable sortable'.split(' '))
found_tables = []

def parse_table(table):
    lines = table.find_all('tr')

    # Extract column names
    colnames = [str(name.find('b').text) for name in lines[0].find_all('td')]
    
    # Simple cleanup of the column names
    colnames = [elem.split('\n')[0] for elem in colnames]
    
    # Parse other lines
    content = []
    for line in lines[1:]:
        content.append([e.text for e in line.find_all('td')])
    
    return pd.DataFrame(data=content, columns=colnames)

for table in content_div.find_all('table') :
    classes_found = set(table['class'])
    if table_class_target <= classes_found:
        # Table to parse
        found_tables.append(parse_table(table))

This scrape all the wikitable from the page, then some cleaning need to be done

## Largest companies

In [13]:
largest_companies = found_tables[0]
largest_companies

,Rank,Name,Headquarters,Revenue,Net income,Employees,Industry,Director
0,1.,Glencore,Baar,119 938,,181 000[1],commodity trading,
1,2.,Nestlé,Vevey,91 075,7 995,253 000,agriculture/food,Peter Brabeck-Letmathe
2,3.,Novartis,Basel,42 455,8 093,90 924,chemical/pharmaceutical,
3,4.,Roche,Basel,35 511,2 390,68 218,chemical/pharmaceutical,
4,5.,ABB,Zurich,29 169,955,104 000,engineering,
5,6.,Adecco,Chéserex,28 534,706,33 000,temp workers,
6,7.,Migros,Zurich,20 385,699,59 934,retail/services,
7,8.,Holcim,Jona,18 468,1 818,59 901,construction materials,
8,9.,Cargill International SA,Geneva,18 275,,475,commodity trading,
9,10.,Coop,Basel,14 901,270,37 370,retail/services,


In [14]:
def set_rank_as_index(df, colname='Rank'):
    if df[colname].dtype==object:
        df[colname] = df[colname].apply(lambda x: int(x.split('.')[0]))
        df.set_index(colname, inplace=True)

set_rank_as_index(largest_companies)
largest_companies

,Name,Headquarters,Revenue,Net income,Employees,Industry,Director
Rank,,,,,,,
1,Glencore,Baar,119 938,,181 000[1],commodity trading,
2,Nestlé,Vevey,91 075,7 995,253 000,agriculture/food,Peter Brabeck-Letmathe
3,Novartis,Basel,42 455,8 093,90 924,chemical/pharmaceutical,
4,Roche,Basel,35 511,2 390,68 218,chemical/pharmaceutical,
5,ABB,Zurich,29 169,955,104 000,engineering,
6,Adecco,Chéserex,28 534,706,33 000,temp workers,
7,Migros,Zurich,20 385,699,59 934,retail/services,
8,Holcim,Jona,18 468,1 818,59 901,construction materials,
9,Cargill International SA,Geneva,18 275,,475,commodity trading,


In [15]:
largest_companies.dtypes

Name            object
Headquarters    object
Revenue         object
Net income      object
Employees       object
Industry        object
Director        object
dtype: object

In [16]:
def custom_to_numeric(column, silently=True):
    if column.dtype != object:
        return column
    
    # Clean cells
    def clean(s: str):
        # Remove spaces
        s = ''.join(s.split())
        
        # Removes reference (`[d]`)
        return s.split('[')[0]
    
    c = column.apply(clean)
    
    # Convert to int
    conversion_force = 'coerc' if silently else 'raise'
    return pd.to_numeric(c, conversion_force)
    
col_to_convert = ['Revenue', 'Net income', 'Employees']

for c in col_to_convert:
    largest_companies[c] = custom_to_numeric(largest_companies[c], False)

largest_companies.dtypes

Name             object
Headquarters     object
Revenue           int64
Net income      float64
Employees       float64
Industry         object
Director         object
dtype: object

Where the fields look well parsed.

## Largest banks
Let do it again for largest banks

In [17]:
largest_banks = found_tables[1]
largest_banks

,Ranked,Name,Headquarters,Total assets,Income,Employees,Manager
0,1.,UBS,Basel,2 060 250,14 690,69 569,"Marcel Ospel, Basler, 2008 abgewählt"
1,2.,Credit Suisse,Zurich,1 339 052,5 850,63 523,
2,3.,Raiffeisen,St. Gallen,108 187,608,6 549,
3,4.,Zürcher Kantonalbank,Zurich,85 914,810,4 276,
4,5.,HSBC Private Bank (Suisse),Geneva,54 012,374,1 929,
5,6.,BNP Paribas (Suisse),Geneva,37 657,309,1 736,
6,7.,Banque cantonale vaudoise,Lausanne,34 875,457,2 349,
7,8.,Julius Baer Group,Zurich,28 648,144,3 427,
8,9.,Banque Migros,Zurich,28 000,87,1 124,
9,10.,Basler Kantonalbank,Basel,24 654,263,1 348,


In [18]:
set_rank_as_index(largest_banks, 'Ranked')
largest_banks

,Name,Headquarters,Total assets,Income,Employees,Manager
Ranked,,,,,,
1,UBS,Basel,2 060 250,14 690,69 569,"Marcel Ospel, Basler, 2008 abgewählt"
2,Credit Suisse,Zurich,1 339 052,5 850,63 523,
3,Raiffeisen,St. Gallen,108 187,608,6 549,
4,Zürcher Kantonalbank,Zurich,85 914,810,4 276,
5,HSBC Private Bank (Suisse),Geneva,54 012,374,1 929,
6,BNP Paribas (Suisse),Geneva,37 657,309,1 736,
7,Banque cantonale vaudoise,Lausanne,34 875,457,2 349,
8,Julius Baer Group,Zurich,28 648,144,3 427,
9,Banque Migros,Zurich,28 000,87,1 124,


In [19]:
col_to_convert = ['Total assets', 'Income', 'Employees']
for c in col_to_convert:
    largest_banks[c] = custom_to_numeric(largest_banks[c])
    
largest_banks

,Name,Headquarters,Total assets,Income,Employees,Manager
Ranked,,,,,,
1,UBS,Basel,2060250,14690,69569,"Marcel Ospel, Basler, 2008 abgewählt"
2,Credit Suisse,Zurich,1339052,5850,63523,
3,Raiffeisen,St. Gallen,108187,608,6549,
4,Zürcher Kantonalbank,Zurich,85914,810,4276,
5,HSBC Private Bank (Suisse),Geneva,54012,374,1929,
6,BNP Paribas (Suisse),Geneva,37657,309,1736,
7,Banque cantonale vaudoise,Lausanne,34875,457,2349,
8,Julius Baer Group,Zurich,28648,144,3427,
9,Banque Migros,Zurich,28000,87,1124,


In [20]:
largest_banks.dtypes

Name            object
Headquarters    object
Total assets     int64
Income           int64
Employees        int64
Manager         object
dtype: object

## And once again for insurances

In [21]:
largest_insurances = found_tables[2]
largest_insurances

,Rang,Name,Headquarters,Ranked by gross inflow of premimus,Net profit,Employees,Manager
0,1.,Zurich Financial Services,Zurich,61 678,4 388,,
1,2.,Swiss Re,Zurich,27 779,1 451,,
2,3.,Winterthur Group,Winterthur,21 269,1 061,,
3,4.,Swiss Life,Zurich,20 211,874,,
4,5.,Bâloise,Basel,6 839,403,,
5,6.,Helvetia Patria,St. Gallen,5 176,301,,
6,7.,Suva,Lucerne,4 149,,,
7,8.,Groupe Allianz (Suisse),,3 631,258,,
8,9.,La Mobilière,Bern,2 608,272,,
9,10.,Vaudoise Assurances,Lausanne,1 955,44,,


In [22]:
set_rank_as_index(largest_insurances, 'Rang')
largest_insurances

,Name,Headquarters,Ranked by gross inflow of premimus,Net profit,Employees,Manager
Rang,,,,,,
1,Zurich Financial Services,Zurich,61 678,4 388,,
2,Swiss Re,Zurich,27 779,1 451,,
3,Winterthur Group,Winterthur,21 269,1 061,,
4,Swiss Life,Zurich,20 211,874,,
5,Bâloise,Basel,6 839,403,,
6,Helvetia Patria,St. Gallen,5 176,301,,
7,Suva,Lucerne,4 149,,,
8,Groupe Allianz (Suisse),,3 631,258,,
9,La Mobilière,Bern,2 608,272,,


In [23]:
col_to_convert = ['Ranked by gross inflow of premimus', 'Net profit']
for c in col_to_convert:
    largest_insurances[c] = custom_to_numeric(largest_insurances[c])
    
largest_insurances

,Name,Headquarters,Ranked by gross inflow of premimus,Net profit,Employees,Manager
Rang,,,,,,
1,Zurich Financial Services,Zurich,61678,4388.0,,
2,Swiss Re,Zurich,27779,1451.0,,
3,Winterthur Group,Winterthur,21269,1061.0,,
4,Swiss Life,Zurich,20211,874.0,,
5,Bâloise,Basel,6839,403.0,,
6,Helvetia Patria,St. Gallen,5176,301.0,,
7,Suva,Lucerne,4149,NaN,,
8,Groupe Allianz (Suisse),,3631,258.0,,
9,La Mobilière,Bern,2608,272.0,,


In [24]:
largest_insurances.dtypes

Name                                   object
Headquarters                           object
Ranked by gross inflow of premimus      int64
Net profit                            float64
Employees                              object
Manager                                object
dtype: object

# Save scrape data

In [25]:
import os

In [30]:
DATA_DIR = 'data'

if not os.path.exists(DATA_DIR):
    print('Create data directory')
    os.mkdir(DATA_DIR)
    
to_save = {
    'largest_companies.csv': largest_companies,
    'largest_insurances.csv': largest_insurances,
    'largest_banks.csv': largest_banks,
}

for k, v in to_save:
    v.to_csv(k)